## Logisitic Regression Model

In [1]:
import findspark
findspark.init()
findspark.find()

from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('LR').getOrCreate()

from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.feature import VectorSlicer, VectorAssembler, ChiSqSelector, VectorIndexer, UnivariateFeatureSelector, VarianceThresholdSelector
from pyspark.sql.functions import *
import numpy as np
from pyspark.sql.types import IntegerType
from pyspark.ml.classification import LogisticRegression
from pyspark.mllib.evaluation import RegressionMetrics


In [2]:
df = spark.read.option("header",True).csv('../../synthea-sample-data/data/10k_synthea_covid19_csv/observations.csv').select('PATIENT','Code', 'Description')
deathDf = spark.read.option("header",True).csv('../../synthea-sample-data/data/10k_synthea_covid19_csv/patients.csv').select('Id', 'DEATHDATE')
deadSet = df.join(deathDf, (df.PATIENT == deathDf.Id)).na.drop().drop('Id', 'Code')
labels = spark.read.option("header",True).csv('../FeatureSelection/dfCovid_DeceasedCovid.csv').select('PATIENT', 'covid-19', 'deceased & covid-19')

merged = df.join(deathDf, (df.PATIENT == deathDf.Id), 'left').drop( 'Id')

merged = merged.withColumn('deceased', when(col('DEATHDATE').isNotNull(), 1)).na.fill(0)
merged = merged.join(labels, ('PATIENT'), 'left').dropDuplicates()

In [3]:
groupedDf = merged.groupBy("PATIENT", 'Code').pivot("Code").agg(count("Code").alias("count")).na.fill(0)
merged =merged.select('PATIENT', 'deceased', 'covid-19', 'deceased & covid-19')
finalDf = groupedDf.join(merged, ['PATIENT'], 'left')
cols = list(set(finalDf.columns) - {'PATIENT', 'deceased', 'Code', 'Description', 'covid-19', 'deceased & covid-19'})
assembler = VectorAssembler().setInputCols(cols).setOutputCol('features')
finalDf = finalDf.withColumn("covid-19", finalDf["covid-19"].cast(IntegerType())).withColumn("deceased & covid-19", finalDf["deceased & covid-19"].cast(IntegerType()))
df = assembler.transform(finalDf)
df.printSchema()

root
 |-- PATIENT: string (nullable = true)
 |-- Code: string (nullable = true)
 |-- 10230-1: long (nullable = true)
 |-- 10480-2: long (nullable = true)
 |-- 10834-0: long (nullable = true)
 |-- 14804-9: long (nullable = true)
 |-- 14959-1: long (nullable = true)
 |-- 1742-6: long (nullable = true)
 |-- 1751-7: long (nullable = true)
 |-- 17861-6: long (nullable = true)
 |-- 18262-6: long (nullable = true)
 |-- 1920-8: long (nullable = true)
 |-- 1960-4: long (nullable = true)
 |-- 1975-2: long (nullable = true)
 |-- 1988-5: long (nullable = true)
 |-- 19926-5: long (nullable = true)
 |-- 19994-3: long (nullable = true)
 |-- 2019-8: long (nullable = true)
 |-- 2028-9: long (nullable = true)
 |-- 20454-5: long (nullable = true)
 |-- 20505-4: long (nullable = true)
 |-- 20565-8: long (nullable = true)
 |-- 20570-8: long (nullable = true)
 |-- 2069-3: long (nullable = true)
 |-- 2075-0: long (nullable = true)
 |-- 2085-9: long (nullable = true)
 |-- 2093-3: long (nullable = true)
 |-- 21

### Chi-Sq Selector

In [4]:
selector = ChiSqSelector(numTopFeatures=10, featuresCol="features",
                         outputCol="selectedFeatures", labelCol="deceased")
chiResult = selector.fit(df).transform(df)

In [5]:
metrics=['accuracy', 'weightedPrecision', 'weightedRecall', 'f1']
metrics_bin=['areaUnderROC', 'areaUnderPR']

def printPerformance(results):
    for m in metrics: print('{}: {:f}'.format(m, my_eval_bin.evaluate(results, {my_eval.metricName: m})))
    for m in metrics_bin: print('{}: {:f}'.format(m, my_eval_bin.evaluate(results, {my_eval_bin.metricName: m})))
    y_test = np.array(results.select("deceased").collect()).flatten()
    y_pred = np.array(results.select("prediction").collect()).flatten()
    print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

Deceased

In [6]:
# stratified split
class0 = df.filter(df["deceased"]==0)
class1 = df.filter(df["deceased"]==1)
print("Class 0 (deceased= 0): ", class0.count())
print("Class 1 (deceased= 1): ", class1.count())
(train, test) = chiResult.randomSplit([0.8, 0.2])

lr = LogisticRegression(maxIter=10, regParam=0.3, elasticNetParam=0.8, labelCol = 'deceased', featuresCol = 'features')
model = lr.fit(train).setFeaturesCol('features')
my_eval = MulticlassClassificationEvaluator(labelCol = 'deceased', predictionCol = 'prediction')
my_eval_bin = BinaryClassificationEvaluator(labelCol ='deceased', rawPredictionCol = 'prediction')
results = model.transform(test)
print("\n[Performance on Deceased Label Test set]")
printPerformance(results)


Class 0 (deceased= 0):  28907916
Class 1 (deceased= 1):  6274051


ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "C:\Users\Mirna Elizondo\anaconda3\lib\site-packages\py4j\clientserver.py", line 475, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "C:\Users\Mirna Elizondo\anaconda3\lib\socket.py", line 704, in readinto
    return self._sock.recv_into(b)
ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Mirna Elizondo\anaconda3\lib\site-packages\py4j\java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "C:\Users\Mirna Elizondo\anaconda3\lib\site-packages\py4j\clientserver.py", line 503, in send_command
    raise Py4JNetworkError(
py4j.protocol.Py4JNetworkError: Error while sending or receiving


ConnectionRefusedError: [WinError 10061] No connection could be made because the target machine actively refused it

Covid-19

In [ ]:
selector = ChiSqSelector(numTopFeatures=10, featuresCol="features",
                         outputCol="selectedFeatures", labelCol="covid-19")
chiResult = selector.fit(df).transform(df)

In [ ]:
def printPerformance(results):
    for m in metrics: print('{}: {:f}'.format(m, my_eval_bin.evaluate(results, {my_eval.metricName: m})))
    for m in metrics_bin: print('{}: {:f}'.format(m, my_eval_bin.evaluate(results, {my_eval_bin.metricName: m})))
    y_test = np.array(results.select("covid-19").collect()).flatten()
    y_pred = np.array(results.select("prediction").collect()).flatten()
    print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))


In [ ]:
(train, test) = chiResult.randomSplit([0.8, 0.2])

lr = LogisticRegression(maxIter=10, regParam=0.3, elasticNetParam=0.8, labelCol = 'covid-19', featuresCol = 'features')
model = lr.fit(train).setFeaturesCol('features')
my_eval = MulticlassClassificationEvaluator(labelCol = 'covid-19', predictionCol = 'prediction')
my_eval_bin = BinaryClassificationEvaluator(labelCol ='covid-19', rawPredictionCol = 'prediction')
results = model.transform(test)
print("\n[Performance on covid-19 Label Test set]")
printPerformance(results)


Deceased & Covid-19

In [ ]:
selector = ChiSqSelector(numTopFeatures=10, featuresCol="features",
                         outputCol="selectedFeatures", labelCol="deceased & covid-19")
chiResult = selector.fit(df).transform(df)

In [ ]:
def printPerformance(results):
    for m in metrics: print('{}: {:f}'.format(m, my_eval_bin.evaluate(results, {my_eval.metricName: m})))
    for m in metrics_bin: print('{}: {:f}'.format(m, my_eval_bin.evaluate(results, {my_eval_bin.metricName: m})))
    y_test = np.array(results.select("deceased & covid-19").collect()).flatten()
    y_pred = np.array(results.select("prediction").collect()).flatten()
    print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))


In [ ]:
(train, test) = chiResult.randomSplit([0.8, 0.2])

lr = LogisticRegression(maxIter=10, regParam=0.3, elasticNetParam=0.8, labelCol = 'deceased & covid-19', featuresCol = 'features')
model = lr.fit(train).setFeaturesCol('features')
my_eval = MulticlassClassificationEvaluator(labelCol = 'deceased & covid-19', predictionCol = 'prediction')
my_eval_bin = BinaryClassificationEvaluator(labelCol ='deceased & covid-19', rawPredictionCol = 'prediction')
results = model.transform(test)
print("\n[Performance on deceased & covid-19 Label Test set]")
printPerformance(results)

### Univariate Selector

Deceased

In [ ]:
selector = UnivariateFeatureSelector(featuresCol="features", outputCol="selectedFeatures",
                                     labelCol="deceased", selectionMode="numTopFeatures")
selector.setFeatureType("continuous").setLabelType("continuous").setSelectionThreshold(10)

uniResult = selector.fit(df).transform(df)

print("UnivariateFeatureSelector output with top %d features selected using f_classif"
      % selector.getSelectionThreshold())
uniResult.select('features', 'selectedFeatures').show()

In [ ]:
metrics=['accuracy', 'weightedPrecision', 'weightedRecall', 'f1']
metrics_bin=['areaUnderROC', 'areaUnderPR']

def printPerformance(results):
    for m in metrics: print('{}: {:f}'.format(m, my_eval_bin.evaluate(results, {my_eval.metricName: m})))
    for m in metrics_bin: print('{}: {:f}'.format(m, my_eval_bin.evaluate(results, {my_eval_bin.metricName: m})))
    y_test = np.array(results.select("deceased").collect()).flatten()
    y_pred = np.array(results.select("prediction").collect()).flatten()
    print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))


In [ ]:
(train, test) = uniResult.randomSplit([0.8, 0.2])
chiResult = selector.fit(df).transform(df)
lr = LogisticRegression(maxIter=10, regParam=0.3, elasticNetParam=0.8, labelCol = 'deceased', featuresCol = 'features')
model = lr.fit(train).setFeaturesCol('features')
my_eval = MulticlassClassificationEvaluator(labelCol = 'deceased', predictionCol = 'prediction')
my_eval_bin = BinaryClassificationEvaluator(labelCol ='deceased', rawPredictionCol = 'prediction')
results = model.transform(test)
print("\n[Performance on deceased & covid-19 Label Test set]")
printPerformance(results)

Covid-19

In [ ]:
selector = UnivariateFeatureSelector(featuresCol="features", outputCol="selectedFeatures",
                                     labelCol="covid-19", selectionMode="numTopFeatures")
selector.setFeatureType("continuous").setLabelType("continuous").setSelectionThreshold(10)

uniResult = selector.fit(df).transform(df)

print("UnivariateFeatureSelector output with top %d features selected using f_classif"
      % selector.getSelectionThreshold())
uniResult.select('features', 'selectedFeatures').show()

In [ ]:
def printPerformance(results):
    for m in metrics: print('{}: {:f}'.format(m, my_eval_bin.evaluate(results, {my_eval.metricName: m})))
    for m in metrics_bin: print('{}: {:f}'.format(m, my_eval_bin.evaluate(results, {my_eval_bin.metricName: m})))
    y_test = np.array(results.select("covid-19").collect()).flatten()
    y_pred = np.array(results.select("prediction").collect()).flatten()
    print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))


In [ ]:
(train, test) = uniResult.randomSplit([0.8, 0.2])
lr = LogisticRegression(maxIter=10, regParam=0.3, elasticNetParam=0.8, labelCol = 'covid-19', featuresCol = 'features')
model = lr.fit(train).setFeaturesCol('features')
my_eval = MulticlassClassificationEvaluator(labelCol = 'covid-19', predictionCol = 'prediction')
my_eval_bin = BinaryClassificationEvaluator(labelCol ='covid-19', rawPredictionCol = 'prediction')
results = model.transform(test)
print("\n[Performance on deceased & covid-19 Label Test set]")
printPerformance(results)

Deceased & Covid-19

In [ ]:
selector = UnivariateFeatureSelector(featuresCol="features", outputCol="selectedFeatures",
                                     labelCol="deceased & covid-19", selectionMode="numTopFeatures")
selector.setFeatureType("continuous").setLabelType("continuous").setSelectionThreshold(10)

uniResult = selector.fit(df).transform(df)

print("UnivariateFeatureSelector output with top %d features selected using f_classif"
      % selector.getSelectionThreshold())
uniResult.select('features', 'selectedFeatures').show()

In [ ]:
def printPerformance(results):
    for m in metrics: print('{}: {:f}'.format(m, my_eval_bin.evaluate(results, {my_eval.metricName: m})))
    for m in metrics_bin: print('{}: {:f}'.format(m, my_eval_bin.evaluate(results, {my_eval_bin.metricName: m})))
    y_test = np.array(results.select("deceased & covid-19").collect()).flatten()
    y_pred = np.array(results.select("prediction").collect()).flatten()
    print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))


In [ ]:
(train, test) = uniResult.randomSplit([0.8, 0.2])
lr = LogisticRegression(maxIter=10, regParam=0.3, elasticNetParam=0.8, labelCol = 'deceased & covid-19', featuresCol = 'features')
model = lr.fit(train).setFeaturesCol('features')
my_eval = MulticlassClassificationEvaluator(labelCol = 'deceased & covid-19', predictionCol = 'prediction')
my_eval_bin = BinaryClassificationEvaluator(labelCol ='deceased & covid-19', rawPredictionCol = 'prediction')
results = model.transform(test)
print("\n[Performance on deceased & covid-19 Label Test set]")
printPerformance(results)